In [11]:
import pandas as pd
import re
from itertools import chain

In [12]:
df = pd.read_csv('1516_1617ChatDataAnalysis.csv', encoding = "ISO-8859-1")

In [13]:
df.head()

started     wait duration  \
0   2/1/17 17:26  0:01:00  0:00:01   
1   2/20/17 7:56  0:01:30  0:00:01   
2   6/1/16 13:05  0:02:49  0:00:01   
3  11/11/15 9:07  0:06:25  0:00:01   
4  8/24/15 15:58  0:10:35  0:00:01   

                                                text  
0  17:26PM 04453099523462400194769663@web.library...  
1  07:56AM +14127098077@twilio.libraryh3lp.com: o...  
2  13:05PM +14436361603@twilio.libraryh3lp.com: o...  
3  09:07AM 30663949299342195104597088@web.library...  
4  15:58PM 59889594109216035951265194@web.library...

In [14]:
content = df['text']
print(content.size)

364


This cell uses regular expressions to separate questions from answers, and remove irrelevant information: "Qt - question by text", "Qw - questions by web", or "An - answers"

Put lists of Q & A's separated out from each record into dictionay, using record numbers as keys. now i can write instructions

In [15]:
content[100]

"15:10PM 98149991129563444342473686@web.libraryh3lp.com: Hi I couldn't find this book \r\r15:11PM 98149991129563444342473686@web.libraryh3lp.com: The Design of Experiments in Neuroscience\r\r15:11PM askandyver2@chat.libraryh3lp.com: did you look in our catalog?\r\r15:11PM askandyver2@chat.libraryh3lp.com: we are having problems with the catalog today\r\r15:11PM askandyver2@chat.libraryh3lp.com: let me check for you\r\r15:11PM 98149991129563444342473686@web.libraryh3lp.com: I searched it online\r\r15:11PM 98149991129563444342473686@web.libraryh3lp.com: thanks\r\r15:14PM askandyver2@chat.libraryh3lp.com: Doesn't look like we own this title.  You can borrow it by using our Interlibrary Loan services:\r\r15:14PM askandyver2@chat.libraryh3lp.com: http://www.library.cmu.edu/using/requesting\r\r15:14PM 98149991129563444342473686@web.libraryh3lp.com: okay thank you for your help good day\r\r15:14PM askandyver2@chat.libraryh3lp.com: by the way is this a text for a class?\r\r15:14PM askandyver2@

In [16]:
## Put QA rounds (cleaned records) in dictionay, with keys being record numbers
def makeDicts(content):
	d = {}
	for index in range(content.size):
		text = content[index]
		QAlist = extractQA(text, index)
		d[index] = QAlist
	return d

In [17]:
## extract each record, remove useless content, keep only the conversation, and order as a list in order that converstation occures
def extractQA(text, index):
	text = text.split("\r\r")
	# print(text)

	patternQt = "[0-9]{2}:[0-9]{2}[AP]M.*@twilio.libraryh3lp.com: (.*)"
	patternQw = "[0-9]{2}:[0-9]{2}[AP]M.*@web.libraryh3lp.com: (.*)"
	patternAn = "[0-9]{2}:[0-9]{2}[AP]M.*@chat.libraryh3lp.com: (.*)" 

	QAlist = []
	for i in range(len(text)):
		QA = re.findall(patternQt, text[i]) or re.findall(patternQw, text[i]) or re.findall(patternAn, text[i])

		if not any("offline message sent" in s for s in QA):
			QAlist.extend(QA)
	# print("lists:", QAlist)
	return QAlist

In [18]:
## get a list of all conversation records
def writeFile(d):

	with open("QAs.txt", "wb") as text_file:
		for key in d: 
			values = d[key]
			# print(values)
			line = (' '.join(values) + '\n\n')
			line = line.encode('utf-8')
			print(line)
			text_file.write(line)

In [19]:
# print records with give start and end record #
def printRecord(start, end):
	for i in range(start,end+1):
		record = d[i]
		print(record)

In [20]:
d = makeDicts(content)
print("lenght of d:", len(d))
printRecord(5,5)
writeFile(d)

lenght of d: 364
['Hello, is there a database for soil properties in the U.S. or globally?', "I was trying to find soil organic content and soil temperature range. There isn't much research paper on the internet for this area, since it's already well measured. ", 'hi .. please email sciref@andrew.cmu.edu for a science librarian', 'Thanks!']
b"Hi, I wanted to checkout a book but am not on campus right now. I will reach tomorrow. Is there a way to prebook or something? This is the book: http://search.library.cmu.edu/vufind/Record/1480587/Holdings#tabnav Let me see if I can find it on the shelf.  We're still unpacking\n\n"
b"Hi, I am doing some research and I found a link which requires access to ProQuest in order to read the article. Does CMU have access to this site, and would I be able to access it off campus? Hi, yes, the Libraries subscribes to Proquest.  Go to the Libraries' homepage www.library.cmu.edu and click on Research Databases (under the search box, also listed Under Search 